In [1]:
import h5py
import pandas as pd

from pathlib import Path

# Prepare paths
RAW_DATA_PATH = Path("data/MillionSongSubset")
OUT_DATA_PATH = Path("data/tracks.csv")

col_names = []
col_names_filled = False
rows = []

# Extract data
for curr_path, _, files in RAW_DATA_PATH.walk():
    for file in files[:1]:
        with h5py.File(curr_path.joinpath(file)) as f:
            values = []
            
            def populate(name, obj: h5py.Dataset):
                global col_names_filled

                if not isinstance(obj, h5py.Group):
                    if not col_names_filled:
                        col_names.append(name)
                    values.append(f[name][:])


            f.visititems(populate)
            rows.append(values)
            col_names_filled = True

In [ ]:
import h5py
import pandas as pd
import json
from pathlib import Path
import os
import tarfile

def extract_file_data(h5_file):
  with h5py.File(h5_file,'r')as f:
   song_data = {
            "artist_mbid": f["metadata/songs"]["artist_mbid"][0].decode('utf-8'),
            "artist_mbtags": [tag.decode('utf-8') for tag in f["musicbrainz/artist_mbtags"]],
            "artist_mbtags_count": list(f["musicbrainz/artist_mbtags_count"]),
            "artist_name": f["metadata/songs"]["artist_name"][0].decode('utf-8'),
            "artist_playmeid": int(f["metadata/songs"]["artist_playmeid"][0]),
            "artist_terms": [term.decode('utf-8') for term in f["metadata/artist_terms"]],
            "artist_terms_freq": list(f["metadata/artist_terms_freq"]),
            "artist_terms_weight": list(f["metadata/artist_terms_weight"]),
            "audio_md5": f["analysis/songs"]["audio_md5"][0].decode('utf-8'),
            "danceability": float(f["analysis/songs"]["danceability"][0]),
            "duration": float(f["analysis/songs"]["duration"][0]),
            "end_of_fade_in": float(f["analysis/songs"]["end_of_fade_in"][0]),
            "energy": float(f["analysis/songs"]["energy"][0]),
            "key": int(f["analysis/songs"]["key"][0]),
            "key_confidence": float(f["analysis/songs"]["key_confidence"][0]),
            "loudness": float(f["analysis/songs"]["loudness"][0]),
            "mode": int(f["analysis/songs"]["mode"][0]),
            "mode_confidence": float(f["analysis/songs"]["mode_confidence"][0]),
            "release": f["metadata/songs"]["release"][0].decode('utf-8'),
            "release_7digitalid": int(f["metadata/songs"]["release_7digitalid"][0]),
            "song_hotttnesss": float(f["metadata/songs"]["song_hotttnesss"][0]),
            "song_id": f["metadata/songs"]["song_id"][0].decode('utf-8'),
            "start_of_fade_out": float(f["analysis/songs"]["start_of_fade_out"][0]),
            "tempo": float(f["analysis/songs"]["tempo"][0]),
            "time_signature": int(f["analysis/songs"]["time_signature"][0]),
            "time_signature_confidence": float(f["analysis/songs"]["time_signature_confidence"][0]),
            "title": f["metadata/songs"]["title"][0].decode('utf-8'),
            "track_7digitalid": int(f["metadata/songs"]["track_7digitalid"][0]),
            "track_id": f["analysis/songs"]["track_id"][0].decode('utf-8'),
            "year": int(f["musicbrainz/songs"]["year"][0])
        }
  return song_data



def process_tar_gz_file(output_csv="data/songs_dataset.csv"):
  extracted_folder ="extracted_folder"
      
  # creating and extracting data from each h5 files
  
  songs_data_list =[]
  for root, _,files in os.walk(extracted_folder):
    for file in files:
      if file.endswith(".h5") :
        h5_path = os.path.join(root,file)
        try:
          song_data = extract_file_data(h5_path)
          songs_data_list.append(song_data)
        except Exception as e :
          print(f"error loading the file {file} : {e}")
          
  
  #  for loading the data into the csv file 
  df = pd.DataFrame(songs_data_list)
  df.to_csv(output_csv , index=False)
  
  
  # display the csv file
  print(f"the output of the file is : {output_csv}")
  
#millionsongsubset.tar.gz
process_tar_gz_file()


the output of the file is : data/songs_dataset.csv


In [77]:
# Create a dataframe and save it
df = pd.DataFrame(rows, columns=col_names)
df.to_csv(OUT_DATA_PATH)

In [81]:
print(df.head(1))

                            analysis/bars_confidence  \
0  [0.643, 0.746, 0.722, 0.095, 0.091, 0.362, 0.4...   

                                 analysis/bars_start  \
0  [0.58521, 2.94247, 5.14371, 7.74554, 10.36149,...   

                           analysis/beats_confidence  \
0  [0.834, 0.851, 0.65, 0.635, 0.532, 0.753, 0.62...   

                                analysis/beats_start  \
0  [0.58521, 1.19196, 1.78893, 2.37813, 2.94247, ...   

                        analysis/sections_confidence  \
0  [1.0, 1.0, 0.218, 0.133, 0.384, 0.326, 0.373, ...   

                             analysis/sections_start  \
0  [0.0, 7.74554, 36.44331, 43.61667, 75.17954, 9...   

                        analysis/segments_confidence  \
0  [0.0, 1.0, 0.483, 0.137, 0.42, 1.0, 0.257, 1.0...   

                      analysis/segments_loudness_max  \
0  [-60.0, -31.646, -34.565, -38.407, -34.696, -2...   

                 analysis/segments_loudness_max_time  \
0  [0.0, 0.10929, 0.11044, 0.0844, 0.058